In [312]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from pylab import rcParams
import random

import seaborn as sb
sb.set_style('darkgrid')


# ML
from sklearn.linear_model import LinearRegression


In [8]:
wild=pd.read_csv('WB_hack_3.csv',header=None)
wild.columns=['utc_event_time', 'utc_event_date', 'user_id', 'city_name', 'ecom_event_action', 'ecom_id', 
              'ecom_brand', 'ecom_variant', 'ecom_currency', 'ecom_price100', 'ecom_qty', 'ecom_grand_total100', 
              'os_manufacturer', 'device_type', 'traffic_src_kind', 'app_version', 'net_type', 'locale']

# EDA и очистка данных

In [232]:
wild.head(5)
#wild.iloc[100:110]

,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,['view_item'],['BEFC96A9D6DEA3A9817BC5E5157B0A1C'],['6B0630D20C913739C8164D8208A1B13D'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[231100],[1],[231100],Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,['add_to_cart'],['BEFC96A9D6DEA3A9817BC5E5157B0A1C'],['6B0630D20C913739C8164D8208A1B13D'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[248500],[1],[248500],Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
2,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,['view_item'],['97F9B1135CF24F4CFFF4664435AEC2B9'],['FA74FF059207A1569295D3E83CA88E53'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[75400],[1],[75400],samsung,smartphone,mobile,4.2.7000,3G,ru
3,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,['view_item'],['97F9B1135CF24F4CFFF4664435AEC2B9'],['FA74FF059207A1569295D3E83CA88E53'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[74500],[1],[74500],samsung,smartphone,mobile,4.2.7000,3G,ru
4,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,['view_item'],['7D9A98C6E8BF033613B74252D9D24B33'],['681425F1391C8CA4026B28066C7B3B4A'],['3B9A6121A509CBADC049AEABD3ED0223'],['RUB'],[131900],[1],[131900],samsung,smartphone,mobile,4.3.1003,4G,ru


In [39]:
wild.shape

(2713346, 18)

* device_type и traffic_src_kind  всего 1 вида. Для построения модели не будем учитывать, но могут пригодиться для создания сервиса и аналитики на основании метрик пользователей
* Данные представлены за 61 день (2 мес)
* Неужели у одного бренда в большинстве случаев будет один продукт?

In [36]:
wild.nunique()

utc_event_time         1963997
utc_event_date              61
user_id                1237662
city_name                 2346
ecom_event_action          419
ecom_id                  39257
ecom_brand               36096
ecom_variant             25271
ecom_currency              242
ecom_price100            43413
ecom_qty                  4765
ecom_grand_total100      44332
os_manufacturer            272
device_type                  1
traffic_src_kind             1
app_version                 56
net_type                     7
locale                       4
dtype: int64

In [233]:
wild.net_type.value_counts()

Wi-Fi       1520746
4G           890634
3G           265236
2G            24812
Other         11884
Ethernet         33
5G                1
Name: net_type, dtype: int64

Всего 5 целевых действий
- view_item	- просмотр
- add_to_cart - добавление в корзину
- begin_checkout - оформление заказа
- purchase - покупка
- remove_from_cart - удаление из корзины

Видим, что во многих строках действия повторяются, посмотрим подробнее

In [115]:
wild['ecom_event_action'].value_counts().reset_index().head(15)

,index,ecom_event_action
0,['view_item'],2201443
1,['add_to_cart'],283371
2,['begin_checkout'],78639
3,['purchase'],50760
4,['remove_from_cart'],19816
5,"['begin_checkout','begin_checkout']",12168
6,"['purchase','purchase']",8580
7,"['begin_checkout','begin_checkout','begin_chec...",8395
8,"['begin_checkout','begin_checkout','begin_chec...",5955
9,"['purchase','purchase','purchase']",5592


In [239]:
# Пример
wild['ecom_event_action'].value_counts().reset_index()['index'][384]

"['begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_checkout','begin_check

Видим, что дублирования возникли не просто так, а одна строка представляет корзину (покупку), которая может состоять из нескольких продуктов (+ их описания и стоимость). Можно сделать анализ по сопутствующим товарам.

In [244]:
wild.iloc[107]

utc_event_time                                       2021-10-21 14:09:23
utc_event_date                                                2021-10-21
user_id                                              9965105300024501182
city_name                                                         Москва
ecom_event_action      ['begin_checkout','begin_checkout','begin_chec...
ecom_id                ['6EB3A8836EBDF486669F65CA40FCB6D9','C0C9F2DC2...
ecom_brand             ['12D89E762536342E35B5F410066B0B45','A1311DE37...
ecom_variant           ['50D05F0ABB80F2B62C5F67F057E88D5F','3B9A6121A...
ecom_currency                                        ['RUB','RUB','RUB']
ecom_price100                                        [38900,15400,23600]
ecom_qty                                                         [1,1,1]
ecom_grand_total100                                  [38900,15400,23600]
os_manufacturer                                                   Xiaomi
device_type                                        

### Очистка данных
Столбцы,  в которых содержится информация о нескольких товарах в корзине
```Python
testw[['ecom_id','ecom_brand','ecom_variant','ecom_currency','ecom_price100','ecom_qty',
      'ecom_grand_total100']]
```

Приведем таблицу к нормальной форме, добавив номер заказа как признак

In [295]:
df=pd.DataFrame()

for label, content in wild.items():
    if label in ['ecom_event_action','ecom_id','ecom_brand','ecom_variant','ecom_currency','ecom_price100','ecom_qty',
      'ecom_grand_total100']:
        a=content.str.translate(str.maketrans({'[': '', ']': '',"'":''})).str.split(',').explode().reset_index()
        df['index']=a['index']
        df[str(label)]=a[str(label)]


columns=['utc_event_time', 'utc_event_date', 'user_id', 'city_name','os_manufacturer', 
         'device_type', 'traffic_src_kind', 'app_version', 'net_type', 'locale']
df=df.merge(wild[columns], right_index=True, left_on='index')

df.rename(columns = {'index': 'order_num_id'}, inplace = True)

In [296]:
df.head(5)

,order_num_id,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,utc_event_time,utc_event_date,user_id,city_name,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,0,view_item,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,231100,1,231100,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,1,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,248500,1,248500,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
2,2,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,75400,1,75400,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,samsung,smartphone,mobile,4.2.7000,3G,ru
3,3,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74500,1,74500,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,samsung,smartphone,mobile,4.2.7000,3G,ru
4,4,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,RUB,131900,1,131900,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,samsung,smartphone,mobile,4.3.1003,4G,ru


Посмотрим датасет заново

In [297]:
df.shape

(3450087, 19)

Теперь видим более репрезентативные данные о брендах и продуктах, общая информация о других признаках продукта также значительно поменялась.

In [298]:
df.nunique()

order_num_id           2713346
ecom_event_action            6
ecom_id                 260335
ecom_brand               44576
ecom_variant              1001
ecom_currency                5
ecom_price100            13120
ecom_qty                    80
ecom_grand_total100      14177
utc_event_time         1963997
utc_event_date              61
user_id                1237662
city_name                 2346
os_manufacturer            272
device_type                  1
traffic_src_kind             1
app_version                 56
net_type                     7
locale                       4
dtype: int64

Видим еще один статус, проверяем, он был только в одном "заказе", строку удалим. 

In [299]:
df.ecom_event_action.value_counts()

view_item           2201443
begin_checkout       725076
add_to_cart          285813
purchase             217935
remove_from_cart      19816
view_item_list            4
Name: ecom_event_action, dtype: int64

In [300]:
df[df.ecom_event_action=='view_item_list']

,order_num_id,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,utc_event_time,utc_event_date,user_id,city_name,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
2549807,2008027,view_item_list,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,3B9A6121A509CBADC049AEABD3ED0223,RUB,1087300,1,1087300,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru
2549808,2008027,view_item_list,D591C1459123DA94B018025503542940,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,RUB,139300,1,139300,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru
2549809,2008027,view_item_list,BD4B5ED1C236588A679299FC428F62C7,75ECD089955F52D949D4BB143AA0F50C957C9413B01995...,3B9A6121A509CBADC049AEABD3ED0223,RUB,138600,1,138600,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru
2549810,2008027,view_item_list,3AF9EEF295A33CECA5FFACBCE2B87A94,B9D2461B6EEE90341EA1027A0FD6110B,3B9A6121A509CBADC049AEABD3ED0223,RUB,728900,1,728900,2021-10-11 07:38:38,2021-10-11,9691711119514764683,NaN,Google,smartphone,mobile,4.3.1003,Wi-Fi,ru


Проверим датасет на пустые значения. 

Пропуск есть только у **городов**: значений много, лучше не удалять, посмотрим, как можно заполнить их.

**Гипотеза:** по связанным признакам (валюта и страна) подставить самое часто встречающееся значение. 

Смотрим  распределение городов (первые 10). Есть перекос в сторону Москвы. Видим Минск. Смотрим валюту (отличных от рубля - 0.001%) и страны (не РФ - 0.05%) - Беларусь и беларуские рубли не встречаются так часто, как Минск, признаки можно вовсе удалить, они не представляют ценности. 

Города встречаются +- равномерно, заменим пустые значения также равномерно.

In [302]:
df.isnull().sum()

order_num_id                0
ecom_event_action           0
ecom_id                     0
ecom_brand                  0
ecom_variant                0
ecom_currency               0
ecom_price100               0
ecom_qty                    0
ecom_grand_total100         0
utc_event_time              0
utc_event_date              0
user_id                     0
city_name              418908
os_manufacturer             0
device_type                 0
traffic_src_kind            0
app_version                 0
net_type                    0
locale                      0
dtype: int64

In [344]:
df.city_name.value_counts().reset_index().head(10)

,index,city_name
0,Москва,644190
1,Санкт-Петербург,195692
2,Екатеринбург,157040
3,Краснодар,129786
4,Казань,98254
5,Минск,81628
6,Новосибирск,80709
7,Самара,69506
8,Нижний Новгород,44969
9,Ростов-на-Дону,44653


In [310]:
df.ecom_currency.value_counts()

RUB    3450051
BYN         26
KZT          6
AMD          3
KGS          1
Name: ecom_currency, dtype: int64

In [307]:
df.locale.value_counts()

ru       3448064
hy          2007
ru-RU         15
en-GB          1
Name: locale, dtype: int64

### Заполняем пропуски, преобразовываем данные, подготавливаем к модели

In [350]:
# using next valid observation to fill na
df.city_name.fillna(method='backfill',inplace=True) 

In [358]:
df=df.drop(df[(df.ecom_currency!='RUB')|((df.locale !='ru')&(df.locale !='ru-RU'))].index)


df.drop(columns=['ecom_currency', 'locale','utc_event_time','device_type','traffic_src_kind'],axis=1,inplace=True)

In [359]:
df

,order_num_id,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_price100,ecom_qty,ecom_grand_total100,utc_event_date,user_id,city_name,os_manufacturer,app_version,net_type
0,0,view_item,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,231100,1,231100,2021-10-01,11090758688966051676,Могилёв,Xiaomi,4.3.4000,Wi-Fi
1,1,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,248500,1,248500,2021-10-01,11090758688966051676,Могилёв,Xiaomi,4.3.4000,Wi-Fi
2,2,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,75400,1,75400,2021-10-01,12487687213536341095,Самара,samsung,4.2.7000,3G
3,3,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,74500,1,74500,2021-10-02,12487687213536341095,Самара,samsung,4.2.7000,3G
4,4,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,131900,1,131900,2021-10-03,10383386994170804641,Москва,samsung,4.3.1003,4G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3450082,2713341,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,1382400,1,1382400,2021-10-10,13331229260693292116,Екатеринбург,samsung,4.2.9000,2G
3450083,2713342,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,1382400,1,1382400,2021-10-10,13331229260693292116,Екатеринбург,samsung,4.2.9000,2G
3450084,2713343,view_item,BDBD23C5359EF7B3F85763C032892B85,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,2309200,1,2309200,2021-10-10,10985970864871842613,Москва,HUAWEI,4.2.5000-huawei,4G
3450085,2713344,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,74900,1,74900,2021-10-10,12409157986873686868,Москва,samsung,4.3.1003,Wi-Fi
